In [2]:
import unittest
import torch
import os.path as osp
import sys

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))


In [3]:
from config import cfg
from model_new_ssa import get_model
from utils.preprocessing import load_img, process_bbox, generate_patch_image, get_iou
from utils.vis import vis_keypoints_with_skeleton, save_obj, render_mesh_orthogonal
from utils.mano import mano

Fix shapedirs bug of MANO


In [3]:
model = get_model(mode='train')
model = torch.nn.DataParallel(model).cuda()
ckpt = torch.load(r"C:\Users\vladi\RP\InterWild\demo\snapshot_6.pth")
load_info = model.load_state_dict(ckpt['network'], strict=False)
model.eval()

Initialize resnet from model zoo
Initialize resnet from model zoo


C:\Users\vladi\AppData\Local\Temp\ipykernel_45980\1470055227.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(r"C:\Users\vladi\RP\InterWild\demo\snapsho

DataParallel(
  (module): Model(
    (body_backbone): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [4]:
from InterHand26M_eval import InterHand26M_eval


In [5]:
from torchvision import transforms

ih_ds = InterHand26M_eval(transform=transforms.ToTensor(), data_split='train', offset=0, size=1000)


In [6]:
ih_ds.total_files()

1361062

In [8]:
from torch.utils.data import DataLoader, Dataset
class MSCOCODataset(Dataset):
    """
    Adapter for MSCOCO dataset that provides the same interface as InterHand2D.
    Only uses validation split.
    """
    def __init__(self, transform=None):
        from MSCOCO_orig import MSCOCO
        self.dataset = MSCOCO(transform=transform, data_split='train')
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        inputs, targets, meta_info = self.dataset[idx]
        
        # Ensure we return the expected format:
        # source_inputs = {'img': tensor}
        # source_targets = {'joint_img', 'bbox' related fields}
        # source_meta_info = validation related fields
        return inputs, targets, meta_info

In [9]:
mscoco_ds = MSCOCODataset()

loading annotations into memory...
Done (t=36.65s)
creating index...
index created!


In [18]:
for i, (inputs, targets, meta_info) in enumerate(ih_ds):
    print(inputs['img'].shape)
    with torch.no_grad():
        out = model({'img': inputs['img'].unsqueeze(0)}, {}, meta_info, 'test')
    print(out['source_hand_feat'].shape)
    break



torch.Size([3, 512, 384])
torch.Size([2, 2048, 8, 8])
